Требуется выполнить следующие действия с данными по облигациям на дату формирования данных:
1.	Оценить рынок рублевых облигаций по распределению купонной доходности, предложить разделение на кластеры по купонной доходности. 
Предложить графическое представление полученных групп/кластеров. (график)
2.	По низкорисковым рублевым облигациям с датой промежуточной выплаты по купонам не более 20% купонного периода назад до текущей даты и облигациям выпущенным в аналогичный период, рассчитать доходность последующих купонов по отобранным облигациям с учетом разницы номинала и цены продажи, а также с учетом налоговых отчислений на период год с текущей дата. (график и/или таблица). 
3.	Допускается обоснованное изменение условий как по критериям оценки рынка, так с учетом выбранной стратегии инвестирования.

Пояснения:
	Для получения текущей даты можно использовать метод  datetime.date.today();
	Период расчета доходности рекомендуется выбирать 365 дней с текущей даты;
	Нужно учитывать накопленный купонный доход (НКД) и разницу котировки и номинала ;
	Расчет доходности можно посмотреть, например, здесь.
(Доход) = (Цена облигации при продаже) + (НКД на момент продажи)
(Расходы) = (Цена облигации при покупке) + (НКД на момент покупки)+(Комиссии за приобретение, реализацию, хранение)
Комиссии Сбера зависят от тарифа, объема сделок и пр. Подробнее см. здесь.
В общем случае считаем налог доход при ставке НДФЛ 13% (при желании можно считать 15%) и комиссию брокера 0.06%
Комиссию Биржи принимаем в 0.0125%. 

Profit =  (((Цена_Номинальная  – Цена_покупки )*(1 ЕСЛИ ≤0│0.87 ЕСЛИ>0)  –  НКД + 0.87*Купонный_доход_руб*X_лет)*100% )/((Цена_покупки+ НКД)*(1+Комиссия брокера в процентах/100 +Комиссия биржи/100))

	Период расчета НКД:  df['COUPONDAYSPASSED']/df['COUPONLENGTH'] , рекомендуется выбирать меньше или равный  0.2. 
Если удовлетворяет условию, полученная величина умножается на COUPONPERCENT/COUPONFREQUENCY.
	Датасет содержит данные, которые нужно преобразовывать в форматы python, а также пустые и искаженные данные, с которыми тоже нужно что-то решить.


In [171]:
import requests
import pandas as pd
import io
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import plotly.express as px

In [172]:
bond_url = 'https://iss.moex.com/iss/apps/infogrid/emission/rates.csv?iss.dp=comma&iss.df=%25d.%25m.%25Y&iss.tf=%25H:%25M:%25S&iss.dtf=%25d.%25m.%25Y%25H:%25M:%25S&iss.only=rates&limit=unlimited&lang=ru'

https://iss.moex.com/iss/reference/783 - КБД
https://iss.moex.com/iss/reference/149 - статические данные?

In [173]:
req = requests.get(bond_url).content

In [174]:
raw_data = pd.read_csv(io.StringIO(req.decode('cp1251')), sep=';', header= 1)
raw_data

,SECID,SHORTNAME,NAME,TYPENAME,ISIN,REGNUMBER,LISTLEVEL,FACEVALUE,FACEUNIT,ISSUESIZE,...,RTL3,RTH3,DISCOUNT1,LIMIT1,DISCOUNT2,LIMIT2,DISCOUNT3,DISCOUNTL0,DISCOUNTH0,FULLCOVERED
0,AMUNIBB2AER2,UBANK11/22,UNIBANK OJSC Series 1,Корпоративные еврооблигации,AMUNIBB2AER2,NaN,3.0,"100,0",USD,10000.0,...,"0,0","12202,06","100,0",NaN,NaN,NaN,NaN,"100,00000","100,00000",1.0
1,AMUNIBB2DER6,UBANK02/24,UNIBANK OJSC Series 1 21,Корпоративные еврооблигации,AMUNIBB2DER6,NaN,3.0,"100,0",USD,33478.0,...,"0,0","12364,62","100,0",NaN,NaN,NaN,NaN,"100,00000","100,00000",1.0
2,AT0000A2UF10,Raiff CAG,Raiffeisen Centrobank AG,Корпоративные еврооблигации,AT0000A2UF10,NaN,3.0,"1000,0",USD,50000.0,...,"0,0","101551,98","100,0",NaN,NaN,NaN,NaN,"100,00000","100,00000",1.0
3,CH0248531110,VTB-24 CHF,VTB CAPITAL S.A. 24 CHF,Корпоративные еврооблигации,CH0248531110,NaN,NaN,"5000,0",CHF,70000.0,...,"0,0","651908,76","100,0",NaN,NaN,NaN,NaN,"100,00000","100,00000",1.0
4,CH0379268748,RZD-23 CHF,RZD Capital PLC 23 CHF,Корпоративные еврооблигации,CH0379268748,NaN,NaN,"5000,0",CHF,90000.0,...,"0,0","398205,36","100,0",NaN,NaN,NaN,NaN,"100,00000","100,00000",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2665,XS2420560869,BCS01/25,BCS SP Plc Series 239,Корпоративные еврооблигации,XS2420560869,NaN,3.0,"100000,0",RUB,700.0,...,"0,0","200000,0","100,0",NaN,NaN,NaN,NaN,"100,00000","100,00000",1.0
2666,XS2423361190,BCS02/27-4,BCS SP Plc Series 240,Корпоративные еврооблигации,XS2423361190,NaN,3.0,"1250,0",USD,8000.0,...,"0,0","151978,1","100,0",NaN,NaN,NaN,NaN,"100,00000","100,00000",1.0
2667,XS2429208486,BCS05/25,BCS SP Plc Series 243,Корпоративные еврооблигации,XS2429208486,NaN,3.0,"1250,0",USD,8000.0,...,"0,0","140873,7","100,0",NaN,NaN,NaN,NaN,"100,00000","100,00000",1.0
2668,XS2439218640,BCS06/25-B,BCS SP Plc Series 244,Корпоративные еврооблигации,XS2439218640,NaN,3.0,"1250,0",USD,8000.0,...,"0,0","148388,18","100,0",NaN,NaN,NaN,NaN,"100,00000","100,00000",1.0


In [175]:
### Предобработка
print('Форма данных:', raw_data.shape[0], 'строк и',raw_data.shape[1],'полей')

Форма данных: 2670 строк и 54 полей


In [176]:
raw_data.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2670 entries, 0 to 2669
Data columns (total 54 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   SECID                   2670 non-null   object 
 1   SHORTNAME               2670 non-null   object 
 2   NAME                    2670 non-null   object 
 3   TYPENAME                2670 non-null   object 
 4   ISIN                    2670 non-null   object 
 5   REGNUMBER               2286 non-null   object 
 6   LISTLEVEL               2399 non-null   float64
 7   FACEVALUE               2670 non-null   object 
 8   FACEUNIT                2670 non-null   object 
 9   ISSUESIZE               2649 non-null   float64
 10  IS_COLLATERAL           2670 non-null   int64  
 11  IS_EXTERNAL             2670 non-null   int64  
 12  PRIMARY_BOARDID         2670 non-null   object 
 13  PRIMARY_BOARD_TITLE     2670 non-null   object 
 14  MATDATE                 2618 non-null   

In [177]:
# Пропущенные наблюдения
raw_data.isna().sum().sort_values(ascending = False).head(10)

IS_RII        2656
SECSUBTYPE    2514
OFFERDATE     2220
DISCOUNT3     1794
LIMIT2        1794
DISCOUNT2     1794
LIMIT1        1794
YIELDATWAP    1517
DURATION      1094
REGNUMBER      384
dtype: int64

In [178]:
# Ищем дубликаты
print(f'Дубликаты: {raw_data.iloc[:,1:].duplicated().sum()}, ({np.round(100*raw_data.iloc[:,1:].duplicated().sum()/len(raw_data),1)}%)')

Дубликаты: 0, (0.0%)


In [179]:
# Ищем пустые переменные
raw_data.nunique().sort_values().head(5)

MORNINGSESSION            1
IS_RII                    1
FULLCOVERED               2
HIGH_RISK                 2
IS_QUALIFIED_INVESTORS    2
dtype: int64

In [180]:
### Исправляем данные

## str, object -> float
# Значение купона должно быть числовым с точкой (float), в наборе данных - str. Из-за запятой

raw_data['COUPONPERCENT'] = raw_data['COUPONPERCENT'].str.replace(',','.').apply(float)
raw_data['COUPONPERCENT'].describe

# FACEVALUE
raw_data['FACEVALUE'] = raw_data['FACEVALUE'].str.replace(',','.').apply(float)
raw_data['FACEVALUE'].describe
# INITIALFACEVALUE
raw_data['INITIALFACEVALUE'] = raw_data['INITIALFACEVALUE'].str.replace(',','.').apply(float)
raw_data['INITIALFACEVALUE'].describe
# PRICE
raw_data['PRICE'] = raw_data['PRICE'].str.replace(',','.').apply(float)
raw_data['PRICE'].describe
# PRICE_RUB
raw_data['PRICE_RUB'] = raw_data['PRICE_RUB'].str.replace(',','.').apply(float)
raw_data['PRICE_RUB'].describe
# COUPONVALUE
raw_data['COUPONVALUE'] = raw_data['COUPONVALUE'].str.replace(',','.').apply(float)
raw_data['COUPONVALUE'].describe


## str, object -> datetime

# ISSUEDATE
raw_data['ISSUEDATE'] = pd.to_datetime(raw_data['ISSUEDATE'],format='%d.%m.%Y')
raw_data['COUPONDATE'] = pd.to_datetime(raw_data['COUPONDATE'],format='%d.%m.%Y')
raw_data['OFFERDATE'] = pd.to_datetime(raw_data['OFFERDATE'],format='%Y-%m-%d')
raw_data['MATDATE'] = pd.to_datetime(raw_data['MATDATE'],format='%d.%m.%Y')
# STARTDATEMOEX # есть пропуски
raw_data['STARTDATEMOEX'] = pd.to_datetime(raw_data['STARTDATEMOEX'],format='%Y-%m-%d')

In [197]:
### Первый подход к расчёту доходности

new_data = raw_data.copy()
new_data['НКД'] = new_data['COUPONVALUE']*(np.divide(new_data['COUPONDAYSPASSED'],new_data['COUPONLENGTH']))
new_data['Годовой доход'] = (new_data['COUPONVALUE']*new_data['COUPONFREQUENCY'])
new_data['Рыночная цена'] = new_data['PRICE']/100*new_data['FACEVALUE']
new_data['Номинал'] = new_data['FACEVALUE']
new_data[['ISIN','PRICE','FACEVALUE','COUPONDATE','COUPONDAYSPASSED','COUPONDAYSREMAIN']].head()
check_date = ((new_data['COUPONDATE'][0])-timedelta(days=new_data['COUPONDAYSREMAIN'][0]))

for i in new_data.index:
    if new_data.loc[i,'OFFERDATE'] < new_data.loc[i,'MATDATE']:
        new_data.loc[i, 'Финальная дата'] = new_data.loc[i,'OFFERDATE']
    else:
        new_data.loc[i,'Финальная дата'] = new_data.loc[i,'MATDATE']


new_data['N оставшихся купонов'] = np.floor(np.divide((new_data['Финальная дата'] - check_date).astype('timedelta64[D]'),new_data['COUPONLENGTH']))
new_data['Валютный риск'] = 0

for i in new_data.index:
    if new_data.loc[i,'FACEUNIT'] != 'RUB':
        new_data.loc[i, 'Валютный риск'] = 1
new_data['COUPONVALUE'].fillna(new_data['COUPONPERCENT']*new_data['FACEVALUE'], inplace = True)
new_data['Годовая купонная доходность'] = ((new_data['COUPONVALUE']*(np.round(np.divide(365,new_data['COUPONFREQUENCY']),0)))/(new_data['FACEVALUE']))
new_data['Срочность в годах'] = (new_data['Финальная дата'] - check_date).astype('timedelta64[M]')/12
new_data['Срочность от выпуска'] = (new_data['Финальная дата'] - new_data['ISSUEDATE']).astype('timedelta64[M]')/12
mat_bins = [-np.inf,0.25, 0.5, 0.75, 1, 2, 3, 4, 5, 7, 10, 15, 20, 30, np.inf]
new_data['Корзина остатка срочности'] = pd.cut(new_data['Срочность в годах'], mat_bins)
new_data['Корзина начальной срочности'] = pd.cut(new_data['Срочность от выпуска'], mat_bins)


In [182]:
new_data[['SECID','COUPONLENGTH','Финальная дата','N оставшихся купонов']].sort_values(by = 'N оставшихся купонов',ascending=False)

,SECID,COUPONLENGTH,Финальная дата,N оставшихся купонов
2499,XS2075963293,92.0,2111-01-01,350.0
960,RU000A1018T2,30.0,2050-05-07,336.0
864,RU000A100ZB9,31.0,2049-09-28,318.0
1558,RU000A102ME7,92.0,2053-07-28,122.0
1557,RU000A102MC1,92.0,2053-07-28,122.0
...,...,...,...,...
2623,XS2361751204,NaN,2024-08-02,NaN
2638,XS2387703866,92.0,NaT,NaN
2641,XS2392969395,92.0,NaT,NaN
2643,XS2393688598,92.0,NaT,NaN


In [199]:
new_data['Корзина остатка срочности'].value_counts()

(1.0, 2.0]      538
(2.0, 3.0]      533
(3.0, 4.0]      305
(-inf, 0.25]    273
(4.0, 5.0]      192
(5.0, 7.0]      145
(0.25, 0.5]     140
(0.5, 0.75]     138
(0.75, 1.0]     121
(7.0, 10.0]     110
(20.0, 30.0]     58
(10.0, 15.0]     42
(15.0, 20.0]     18
(30.0, inf]       5
Name: Корзина остатка срочности, dtype: int64

In [200]:
new_data['Корзина начальной срочности'].value_counts()

(4.0, 5.0]      608
(2.0, 3.0]      520
(5.0, 7.0]      406
(7.0, 10.0]     328
(3.0, 4.0]      250
(10.0, 15.0]    108
(1.0, 2.0]       89
(20.0, 30.0]     66
(15.0, 20.0]     41
(30.0, inf]      27
(0.75, 1.0]      20
(0.25, 0.5]       2
(-inf, 0.25]      2
(0.5, 0.75]       0
Name: Корзина начальной срочности, dtype: int64

new_data_cluster = new_data[['LISTLEVEL','IS_QUALIFIED_INVESTORS','Валютный риск','HIGH_RISK','Годовая купонная доходность','Срочность в месяцах']]
new_data_cluster = new_data_cluster.dropna()
#for i in range(1,10):
    #kmeanModel = KMeans(n_clusters=i)
    #kmeanModel.fit(new_data_cluster)

kmeans = KMeans(n_clusters = 6)
new_data_cluster['Кластер'] = kmeans.fit_predict(new_data_cluster)
new_data_cluster['Кластер'] = new_data_cluster['Кластер'].astype('category')
sns.relplot(
    x="Срочность в месяцах", y="Годовая купонная доходность", hue="Кластер", data=new_data_cluster, height=6,
)

In [183]:
### Пишем функцию для расчёта доходности
# https://journal.open-broker.ru/investments/formula-dlya-rascheta-dohodnosti-obligacij-k-pogasheniyu/
### входные параметры:
# df
# nkd_col - поле с накопленным купонным доходом в dataframe, в абсолютном выражении
# coup_val_col - поле с купонными доходами в абсолютном выражении
# market_price_col - поле с текущей рыночной ценой облигации, в абсолютном выражении
# nom_col - поле со значением номинала, в абсолютном выражении
# broker_fee_rate - ставка комиссии брокера, в процентах
# ex_fee_rate  - ставка комиссии биржи, в процентах
# tax_rate - ставка налога, в десятичных долях
# coupons_n_col - количество оставшихся купонов, в штуках
# final_date_col - дата оферты или погашения бумаги
def calc_bond_yield(
    df,
    nkd_col,
    coup_val_col,
    market_price_col,
    nom_col,
    coupons_n_col,
    broker_fee_rate,
    ex_fee_rate,
    tax_rate,
    final_date_col,
    check_date
    ):
    df['REALBONDYIELD'] = 0
    for i in df.index:
        if (df.loc[i,nom_col]>df.loc[i,market_price_col]):
            df.loc[i,'REALBONDYIELD'] = 100*np.divide(np.divide(((
                df.loc[i, nom_col] - df.loc[i, market_price_col])*(1-tax_rate) - df.loc[i,nkd_col] + (
                (1-tax_rate)*df.loc[i,coup_val_col]*(df.loc[i,coupons_n_col])    
                )),(
                    df.loc[i,market_price_col]+df.loc[i,nkd_col])*(1+broker_fee_rate/100+ex_fee_rate/100
                )),(df.loc[i,final_date_col] - check_date).days/365)
        else:
            df.loc[i,'REALBONDYIELD'] = 100*np.divide(np.divide(((
                df.loc[i, nom_col] - df.loc[i, market_price_col]) - df.loc[i,nkd_col] + (
                (1-tax_rate)*df.loc[i,coup_val_col]*df.loc[i,coupons_n_col]    
                )),(
                    df.loc[i,market_price_col]+df.loc[i,nkd_col])*(1+(broker_fee_rate/100)+(ex_fee_rate/100)
                )),(df.loc[i,final_date_col] - check_date).days/365)

In [184]:
calc_bond_yield(
    df = new_data, 
    nkd_col = 'НКД',
    coup_val_col = 'COUPONVALUE',
    market_price_col = 'Рыночная цена',
    nom_col = 'Номинал',
    coupons_n_col = 'N оставшихся купонов',
    broker_fee_rate = 0.06,
    ex_fee_rate = 0.0125,
    tax_rate = 0.13,
    final_date_col = 'Финальная дата',
    check_date = check_date
    )

C:\Users\mikha\AppData\Local\Temp\ipykernel_8144\3824571069.py:37: RuntimeWarning:

divide by zero encountered in divide

C:\Users\mikha\AppData\Local\Temp\ipykernel_8144\3824571069.py:30: RuntimeWarning:

divide by zero encountered in divide



In [185]:
new_data.dropna(subset = ['Годовой доход']).sort_values(by = 'REALBONDYIELD', ascending=False)

,SECID,SHORTNAME,NAME,TYPENAME,ISIN,REGNUMBER,LISTLEVEL,FACEVALUE,FACEUNIT,ISSUESIZE,...,НКД,Годовой доход,Рыночная цена,Номинал,Финальная дата,N оставшихся купонов,Валютный риск,Годовая купонная доходность,Срочность в годах,REALBONDYIELD
590,RU000A0ZZL05,МетИнвБ Б3,Металлинвестбанк ПАО АКБ БО-03,Биржевые облигации,RU000A0ZZL05,4B020302440B,3.0,1000.0,RUB,2700000.0,...,0.504066,91.740000,949.80,1000.0,2022-09-07,0.0,0,8.348340,0.000000,inf
2480,XS2038593419,BCS09/22,BCS SP Plc Series 52,Корпоративные еврооблигации,XS2038593419,NaN,3.0,1250.0,USD,8000.0,...,87.260273,176.438354,478.50,1250.0,2022-09-09,0.0,1,12.844712,0.000000,1.882294e+04
816,RU000A100V61,ОРГрупп1Р1,ОР ООО 001P-01,Биржевые облигации,RU000A100V61,4B02-01-00412-R-001P,3.0,250.0,RUB,1483150.0,...,6.247033,29.920000,74.75,250.0,2022-09-22,0.0,0,2.722720,0.000000,4.389610e+03
1055,RU000A101KJ2,Калита1P1,Калита 001Р-01,Биржевые облигации,RU000A101KJ2,4B02-01-00524-R,3.0,1000.0,RUB,300000.0,...,8.220000,147.960000,32.40,1000.0,2023-09-12,12.0,0,0.369900,1.000000,2.335365e+03
113,RU000A0JUHV4,ВейлФин1,Вейл Финанс ООО 01 обл.,Корпоративные облигации,RU000A0JUHV4,4-01-36435-R,3.0,1000.0,RUB,8000000.0,...,0.000000,99.720000,78.70,1000.0,2023-03-08,1.0,0,9.074520,0.416667,2.151503e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2572,XS2260179762,E-H-23,E-House (China) 10/06/23,Корпоративные еврооблигации,XS2260179762,NaN,NaN,1000.0,USD,200000.0,...,18.480874,76.000000,NaN,1000.0,2023-06-10,1.0,1,6.916000,0.750000,NaN
2574,XS2263765856,UZB-30,Republic of Uzbekistan 11/30,Облигации государств,XS2263765856,NaN,NaN,1000.0,USD,555000.0,...,10.557065,37.000000,NaN,1000.0,2030-11-25,16.0,1,3.367000,8.166667,NaN
2638,XS2387703866,TCS-perp2,TCS Finance DAC UNDT,Корпоративные еврооблигации,XS2387703866,NaN,3.0,1000.0,USD,600000.0,...,12.880435,60.000000,841.20,1000.0,NaT,NaN,1,1.365000,NaN,NaN
2641,XS2392969395,CBOM Finan,CBOM Finance P.L.C. 7.625 UNDT,Корпоративные еврооблигации,XS2392969395,NaN,3.0,1000.0,USD,350000.0,...,13.468071,76.250000,710.60,1000.0,NaT,NaN,1,1.734687,NaN,NaN


kbd_url = 'https://iss.moex.com/iss/history/engines/stock/zcyc/yearyields.json'
kbd_req = requests.get(kbd_url).content
kbd_data = pd.read_json(io.StringIO(kbd_req).decode('cp1251'), lines = True)
#kbd_data = pd.json_normalize(kbd_data, max_level = 0)

kbd_alt = requests.get('https://www.cbr.ru/hd_base/zcyc_params/zcyc/?DateTo=07.09.2022').content
kbd_alt

In [190]:
fig = px.histogram(new_data[(new_data['REALBONDYIELD'] < 100)&(new_data['REALBONDYIELD'] > -10)], x = 'REALBONDYIELD')
fig.show()

In [187]:
new_data[new_data['TYPENAME'] == 'Корпоративные облигации'].dropna(subset = ['Годовой доход'])

,SECID,SHORTNAME,NAME,TYPENAME,ISIN,REGNUMBER,LISTLEVEL,FACEVALUE,FACEUNIT,ISSUESIZE,...,НКД,Годовой доход,Рыночная цена,Номинал,Финальная дата,N оставшихся купонов,Валютный риск,Годовая купонная доходность,Срочность в годах,REALBONDYIELD
13,RU000A0JQ7Z2,РЖД-19 обл,"""Российские ЖД"" ОАО 19 обл.",Корпоративные облигации,RU000A0JQ7Z2,4-19-65045-D,2.0,1000.00,RUB,10000000.0,...,12.473187,78.28,996.500,1000.00,2024-07-08,3.0,0,7.123480,1.833333,5.003005
14,RU000A0JQAL8,ДОМ.РФ14об,ДОМ.РФ (АО) обл. сер. А14,Корпоративные облигации,RU000A0JQAL8,4-14-00739-A,1.0,1000.00,RUB,7000000.0,...,51.987500,166.36,1023.700,1000.00,2023-05-15,1.0,0,15.138760,0.666667,-0.450410
15,RU000A0JQAM6,ДОМ.РФ15об,ДОМ.РФ (АО) обл. сер. А15,Корпоративные облигации,RU000A0JQAM6,4-15-00739-A,1.0,1000.00,RUB,7000000.0,...,96.436522,201.64,1099.200,1000.00,2028-09-15,11.0,0,18.349240,6.000000,10.666002
16,RU000A0JQRD9,РЖД-23 обл,"""Российские ЖД"" ОАО 23 обл.",Корпоративные облигации,RU000A0JQRD9,4-23-65045-D,2.0,1000.00,RUB,15000000.0,...,10.322637,78.28,995.800,1000.00,2025-01-16,4.0,0,7.123480,2.333333,5.447773
17,RU000A0JR4U9,ГлДорога 3,Главная дорога АО об.сер. 03,Корпоративные облигации,RU000A0JR4U9,4-03-12755-A,2.0,875.00,RUB,8000000.0,...,50.119863,60.21,769.825,875.00,2028-10-30,6.0,0,25.116171,6.083333,7.047496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,RU000A104K37,СибЭнМаш01,Сибэнергомаш - БКЗ 01,Корпоративные облигации,RU000A104K37,4-01-00604-R,3.0,1000.00,RUB,300000.0,...,8.219780,149.60,1000.100,1000.00,2025-02-13,9.0,0,3.403400,2.416667,11.563945
2160,RU000A104SJ9,БДеньги-03,МФК Быстроденьги 03,Корпоративные облигации,RU000A104SJ9,4-01-00487-R-001P,3.0,1000.00,RUB,132085.0,...,1.780000,128.16,980.900,1000.00,2025-04-19,31.0,0,0.320400,2.583333,11.771406
2178,RU000A104V00,ПионЛизБР5,Пионер-Лизинг БР5,Корпоративные облигации,RU000A104V00,4-01-00331-R,3.0,1000.00,RUB,400000.0,...,1.132000,203.76,999.800,1000.00,2032-04-15,116.0,0,0.509400,9.583333,17.790413
2182,RU000A104VD6,СФО СФ 01,СФО Спутник Фин 01,Корпоративные облигации,RU000A104VD6,4-01-00655-R,3.0,1000.00,RUB,440000.0,...,13.699176,398.92,1000.000,1000.00,2027-07-13,4.0,0,9.075430,4.833333,6.776584


In [188]:
new_data.TYPENAME.value_counts()

Биржевые облигации             1639
Корпоративные еврооблигации     353
Корпоративные облигации         306
Коммерческая облигация          151
Региональные облигации          114
ОФЗ                              58
Облигации государств             26
Еврооблигации МинФина            14
Муниципальные облигации           9
Name: TYPENAME, dtype: int64